<a href="https://colab.research.google.com/github/rezaafsharniakan/DeepLEarningForComputerVision_python/blob/main/Neural_style_transfer_in_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Getting the style and content images

from tensorflow import keras
base_image_path = keras.utils.get_file(
  "sf.jpg", origin="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQZzMLVR8a934zvadogkkmipraRAvV_ofIzbw&usqp=CAU")
style_reference_image_path = keras.utils.get_file(
  "starry_night.jpg",
  origin="https://bloximages.newyork1.vip.townnews.com/myleaderpaper.com/content/tncms/assets/v3/editorial/8/65/8652cf99-0f7d-556d-8851-fb91429949ac/63d538a1bcad7.image.jpg")
original_width, original_height = keras.utils.load_img(base_image_path).size
img_height = 400
img_width = round(original_width * img_height / original_height)

261383/261383 [==============================] - 0s 0us/step


In [2]:
#Auxiliary functions

import numpy as np
def preprocess_image(image_path):
    img = keras.utils.load_img(
    image_path, target_size=(img_height, img_width))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg19.preprocess_input(img)
    return img
def deprocess_image(img):
    img = img.reshape((img_height, img_width, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype("uint8")
    return img

In [3]:
#Using a pretrained VGG19 model to create a feature extractor

model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False)
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

80134624/80134624 [==============================] - 1s 0us/step


In [4]:
# Content loss
def content_loss(base_img, combination_img):
    return tf.reduce_sum(tf.square(combination_img - base_img))

In [5]:
# Style loss
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram
def style_loss(style_img, combination_img):
    S = gram_matrix(style_img)
    C = gram_matrix(combination_img)
    channels = 3
    size = img_height * img_width
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [6]:
# Total variation loss
def total_variation_loss(x):
    a = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :]
    )
    b = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))

In [7]:
# Defining the final loss that you’ll minimize
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layer_name = "block5_conv2"
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8
def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0)
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(
        style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * style_loss_value
    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

In [8]:
 #Setting up the gradient-descent process

import tensorflow as tf
@tf.function
def compute_loss_and_grads(
    combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads
optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96))

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))
iterations = 4000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(combination_image, base_image, style_reference_image)
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print(f"Iteration {i}: loss={loss:.2f}")
        img = deprocess_image(combination_image.numpy())
        fname = f"combination_image_at_iteration_{i}.png"
        keras.utils.save_img(fname, img)

Iteration 100: loss=6353.59
Iteration 200: loss=5114.70
Iteration 300: loss=4553.68
Iteration 400: loss=4220.45
Iteration 500: loss=3995.90
Iteration 600: loss=3831.66
Iteration 700: loss=3705.19
Iteration 800: loss=3603.95
Iteration 900: loss=3520.76
Iteration 1000: loss=3450.90
Iteration 1100: loss=3391.15
Iteration 1200: loss=3339.48
Iteration 1300: loss=3294.18
Iteration 1400: loss=3254.21
Iteration 1500: loss=3218.83
Iteration 1600: loss=3187.27
Iteration 1700: loss=3158.82
Iteration 1800: loss=3133.11
Iteration 1900: loss=3109.73
Iteration 2000: loss=3088.38
Iteration 2100: loss=3068.81
Iteration 2200: loss=3050.85
Iteration 2300: loss=3034.28
Iteration 2400: loss=3018.98
Iteration 2500: loss=3004.84
Iteration 2600: loss=2991.74
Iteration 2700: loss=2979.58
Iteration 2800: loss=2968.24
Iteration 2900: loss=2957.67
Iteration 3000: loss=2947.80
Iteration 3100: loss=2938.56
Iteration 3200: loss=2929.89
Iteration 3300: loss=2921.74
Iteration 3400: loss=2914.10
Iteration 3500: loss=29